# Wine Distillation - Leverage outputs of a large model to distill a smaller one

OpenAI recently released Distillation which allows to leverage the outputs of a (large) model to fine-tune another (smaller) model. This can significantly reduce the price and the latency for specific tasks as you move to a smaller model. In this exercise we'll look at a dataset, distill the output of `gpt-4o` to `gpt-4o-mini` and show how we can get significantly better results than on a generic, non-distilled, `4o-mini`.



## Overview

This notebook contains three sections: 
1. **Assessing a baseline**: Evaluating an out of the box `gpt-4o-mini` and `gpt-4o` models and understand performance
3. **Distillation**: Store the good completions and create a dataset for fine tuning your smaller model. 
4. **Extension**: If you finished the exercise and still have some time, you can try these ideas! 

## 1. Assessing a baseline for funtion calling 

When Fine Tuning a model, it's important to understand what your starting point is. For this exercise we'll be using this [Wine Reviews Dataset](https://www.kaggle.com/datasets/zynicide/wine-reviews) from Kaggle. This dataset has a large number of rows and you're free to run this cookbook on the whole data, but to speed things up, I'll narrow down the dataset to only French wine to focus on less rows and grape varieties.

We're looking at a classification problem where we'd like to guess the grape variety based on all other criterias available, including description, subregion and province that we'll include in the prompt. It gives a lot of information to the model, you're free to also remove some information that can help significantly the model such as the region in which it was produced to see if it does a good job at finding the grape.

In [1]:
%pip install --upgrade openai -q


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import openai
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import concurrent.futures


client = openai.OpenAI()

In [3]:
df = pd.read_csv('data/winemag-data-130k-v2.csv')
df_france = df[df['country'] == 'France']

# Let's also filter out wines that have less than 5 references with their grape variety – even though we'd like to find those
# they're outliers that we don't want to optimize for that would make our enum list be too long
# and they could also add noise for the rest of the dataset on which we'd like to guess, eventually reducing our accuracy.

varieties_less_than_five_list = df_france['variety'].value_counts()[df_france['variety'].value_counts() < 5].index.tolist()
df_france = df_france[~df_france['variety'].isin(varieties_less_than_five_list)]

df_france_subset = df_france.sample(n=500)
df_france_subset.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
84154,84154,France,"The wine has a lively mousse, crispened with a...",Oriane Brut,89,43.0,Champagne,Champagne,NaN,Roger Voss,@vossroger,Faniel et Fils NV Oriane Brut (Champagne),Champagne Blend,Faniel et Fils
52435,52435,France,"While Négrette dominates this blend, the addit...",Quintessence,90,25.0,Southwest France,Fronton,NaN,Roger Voss,@vossroger,Château Coutinel 2011 Quintessence Red (Fronton),Red Blend,Château Coutinel
53612,53612,France,"This wine is rich and full bodied, already del...",Grand Fumé,89,34.0,Loire Valley,Pouilly-Fumé,NaN,Roger Voss,@vossroger,Fournier Père et Fils 2013 Grand Fumé (Pouill...,Sauvignon Blanc,Fournier Père et Fils
11028,11028,France,"A fruity, lightly tannic blend that comes from...",Chevalier du Grand Loc,86,10.0,Southwest France,Côtes de Duras,NaN,Roger Voss,@vossroger,Vincent Lataste 2010 Chevalier du Grand Loc Re...,Bordeaux-style Red Blend,Vincent Lataste
63627,63627,France,"While showing some smoky wood, this wine is mo...",Genevrières Premier Cru,93,95.0,Burgundy,Meursault,NaN,Roger Voss,@vossroger,Bouchard Père & Fils 2011 Genevrières Premier ...,Chardonnay,Bouchard Père & Fils


In [4]:
varieties = np.array(df_france['variety'].unique()).astype('str')
varieties

array(['Gewürztraminer', 'Pinot Gris', 'Gamay',
       'Bordeaux-style White Blend', 'Champagne Blend', 'Chardonnay',
       'Petit Manseng', 'Riesling', 'White Blend', 'Pinot Blanc',
       'Alsace white blend', 'Bordeaux-style Red Blend', 'Malbec',
       'Tannat-Cabernet', 'Rhône-style Red Blend', 'Ugni Blanc-Colombard',
       'Savagnin', 'Pinot Noir', 'Rosé', 'Melon',
       'Rhône-style White Blend', 'Pinot Noir-Gamay', 'Colombard',
       'Chenin Blanc', 'Sylvaner', 'Sauvignon Blanc', 'Red Blend',
       'Chenin Blanc-Chardonnay', 'Cabernet Sauvignon', 'Cabernet Franc',
       'Syrah', 'Sparkling Blend', 'Duras', 'Provence red blend',
       'Tannat', 'Merlot', 'Malbec-Merlot', 'Chardonnay-Viognier',
       'Cabernet Franc-Cabernet Sauvignon', 'Muscat', 'Viognier',
       'Picpoul', 'Altesse', 'Provence white blend', 'Mondeuse',
       'Grenache-Syrah', 'G-S-M', 'Pinot Meunier', 'Cabernet-Syrah',
       'Vermentino', 'Marsanne', 'Colombard-Sauvignon Blanc',
       'Gros and Peti

### Generating the prompt

In [5]:
def generate_prompt(row, varieties):
    # Format the varieties list as a comma-separated string
    variety_list = ', '.join(varieties)
    
    prompt = f"""
    Based on this wine review, guess the grape variety:
    This wine is produced by {row['winery']} in the {row['province']} region of {row['country']}.
    It was grown in {row['region_1']}. It is described as: "{row['description']}".
    The wine has been reviewed by {row['taster_name']} and received {row['points']} points.
    The price is {row['price']}.

    Here is a list of possible grape varieties to choose from: {variety_list}.
    
    What is the likely grape variety? Answer only with the grape variety name or blend from the list.
    """
    return prompt

# Example usage with a specific row
prompt = generate_prompt(df_france.iloc[0], varieties)
prompt

'\n    Based on this wine review, guess the grape variety:\n    This wine is produced by Trimbach in the Alsace region of France.\n    It was grown in Alsace. It is described as: "This dry and restrained wine offers spice in profusion. Balanced with acidity and a firm texture, it\'s very much for food.".\n    The wine has been reviewed by Roger Voss and received 87 points.\n    The price is 24.0.\n\n    Here is a list of possible grape varieties to choose from: Gewürztraminer, Pinot Gris, Gamay, Bordeaux-style White Blend, Champagne Blend, Chardonnay, Petit Manseng, Riesling, White Blend, Pinot Blanc, Alsace white blend, Bordeaux-style Red Blend, Malbec, Tannat-Cabernet, Rhône-style Red Blend, Ugni Blanc-Colombard, Savagnin, Pinot Noir, Rosé, Melon, Rhône-style White Blend, Pinot Noir-Gamay, Colombard, Chenin Blanc, Sylvaner, Sauvignon Blanc, Red Blend, Chenin Blanc-Chardonnay, Cabernet Sauvignon, Cabernet Franc, Syrah, Sparkling Blend, Duras, Provence red blend, Tannat, Merlot, Malbec

### Preparing functions to Store Completions

As we're looking at a limited list of response (enumerate list of grape varieties), let's leverage structured outputs so we make sure the model will answer from this list. This also allows us to compare the model's answer directly with the grape variety and have a deterministic answer (compared to a model that could answer "I think the grape is Pinot Noir" instead of just "Pinot noir"), on top of improving the performance to avoid grape varieties not in our dataset.

If you want to know more on Structured Outputs you can read this cookbook and this documentation guide.

In [6]:
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "grape-variety",
        "schema": {
            "type": "object",
            "properties": {
                "variety": {
                    "type": "string",
                    "enum": varieties.tolist()
                }
            },
            "additionalProperties": False,
            "required": ["variety"],
        },
        "strict": True
    }
}

In [7]:
# Initialize the progress index
metadata_value = "wine-distillation" # that's a funny metadata tag :-)

# Function to call the API and process the result for a single model (blocking call in this case)
def call_model(model, prompt):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You're a sommelier expert and you know everything about wine. You answer precisely with the name of the variety/blend."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        response_format=response_format,
        store=True,
        metadata={
            "distillation": metadata_value,
        },
    )
    return json.loads(response.choices[0].message.content.strip())['variety']

### Parallel Processing

In [8]:
def process_example(index, row, model, df, progress_bar):
    global progress_index

    try:
        # Generate the prompt using the row
        prompt = generate_prompt(row, varieties)

        df.at[index, model + "-variety"] = call_model(model, prompt)
        
        # Update the progress bar
        progress_bar.update(1)
        
        progress_index += 1
    except Exception as e:
        print(f"Error processing model {model}: {str(e)}")

def process_dataframe(df, model):
    global progress_index
    progress_index = 1  # Reset progress index

    # Create a tqdm progress bar
    with tqdm(total=len(df), desc="Processing rows") as progress_bar:
        # Process each example concurrently using ThreadPoolExecutor
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = {executor.submit(process_example, index, row, model, df, progress_bar): index for index, row in df.iterrows()}
            
            for future in concurrent.futures.as_completed(futures):
                try:
                    future.result()  # Wait for each example to be processed
                except Exception as e:
                    print(f"Error processing example: {str(e)}")

    return df

In [10]:
df_france_subset = process_dataframe(df_france_subset, "gpt-4o")

Processing rows: 100%|██████████| 500/500 [00:27<00:00, 18.51it/s]


In [11]:
df_france_subset = process_dataframe(df_france_subset, "gpt-4o-mini")

Processing rows: 100%|██████████| 500/500 [00:31<00:00, 15.91it/s]


### Comparing gpt-4o and gpt-4o-mini

Now that we've got all chat completions for those two models ; let's compare them against the expected grape variety and assess their accuracy at finding it. We'll do this directly in python here as we've got a simple string check to run, but if your task involves more complex evals you can leverage OpenAI Evals or our open-source eval framework.


In [12]:
models = ['gpt-4o', 'gpt-4o-mini']

def get_accuracy(model, df):
    return np.mean(df['variety'] == df[model + '-variety'])

for model in models:
    print(f"{model} accuracy: {get_accuracy(model, df_france_subset) * 100:.2f}%")

gpt-4o accuracy: 77.60%
gpt-4o-mini accuracy: 62.80%


We can see that gpt-4o is better a finding grape variety than 4o-mini (12% - 20% higher relatively to 4o-mini!). Now I'm wondering if we're making gpt-4o drink wine during training!

## 2. Distillation 

It is very clear that 4o performs better than 4o-mini for this type of task. Now let's see if we can use our completions for 4o to distill the 4o-mini model. [This section](https://platform.openai.com/docs/guides/evals/generate-datasets-from-real-traffic) of our docs shows how you can start storing your completions. 

In [14]:
# copy paste your fine-tune job ID below
finetune_job = client.fine_tuning.jobs.retrieve("ftjob-y4AcDdUnnMk59fMHmQc6Vexj")

if finetune_job.status == 'succeeded':
    fine_tuned_model = finetune_job.fine_tuned_model
    print('finetuned model: ' + fine_tuned_model)
else:
    print('finetuned job status: ' + finetune_job.status)

finetuned model: ft:gpt-4o-mini-2024-07-18:distillation-test:wine-test:AKizvB1f


Now you need to go into your [ChatCompletions](https://platform.openai.com/chat-completions) front end, create your dataset and start a distillation process. 

In [15]:
validation_dataset = df_france.sample(n=300)

models.append(fine_tuned_model)

for model in models:
    another_subset = process_dataframe(validation_dataset, model)

Processing rows: 100%|██████████| 300/300 [00:24<00:00, 12.24it/s]


Let's compare the accuracy of the models. 

In [16]:
for model in models:
    print(f"{model} accuracy: {get_accuracy(model, another_subset) * 100:.2f}%")

gpt-4o accuracy: 81.33%
gpt-4o-mini accuracy: 68.67%
ft:gpt-4o-mini-2024-07-18:distillation-test:wine-test:AKizvB1f accuracy: 83.00%


## 3. Extentions 

If you've already completed the execise above, congratulations! Here are a few ideas on how to turn this into a more exciting project: 

1. Compare distillation with simple fine tuning. Anything you're noticing? 
2. Build a RAG system that can give you wine recommendations based on a customer request. 
3. Build a front end application where you can visualise this chatbot. 

## Conclusion 

Congratulations on getting this far! Now you have a better understanding of what distillation is. You can think about more usecases where distillation may be useful. Keen to see what you'll be building! 